In [23]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from stockPublicFunction import *
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, accuracy_score

In [24]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
import numpy as np

def analyze_stock(stock_name, stock_code, stock_type, revenue_per_share_yoy, price_data, revenue_per_share,
                  PB, revenue_t3m_avg, revenue_t3m_yoy, majority_shareholders_share_ratio, total_shareholders_count,
                  latest_close_price):
    """分析股票数据"""

    # 创建有效数据列表
    valid_data = [
        (revenue, price, rev_per_share)
        for revenue, price, rev_per_share in zip(revenue_t3m_yoy, price_data, revenue_per_share)
        if None not in (revenue, price, rev_per_share) and not (np.isnan(revenue) or np.isnan(price) or np.isnan(rev_per_share))
    ]

    if not valid_data:
        return None

    # 解包有效数据
    valid_revenue, valid_price, valid_rev_per_share = zip(*valid_data)

    # 对数据进行样条插值
    interpolated_revenue = spline_interpolation(np.array(valid_revenue))
    interpolated_price = spline_interpolation(np.array(valid_price))
    interpolated_rev_per_share = spline_interpolation(np.array(valid_rev_per_share))

    # 准备时间序列数据
    price_series = interpolated_price.reshape(-1, 1)
    revenue_series = interpolated_revenue.reshape(-1, 1)
    rev_per_share_series = interpolated_rev_per_share.reshape(-1, 1)

    # 正规化与归一化数据
    revenue_normalized, _, scaler_X1 = normalize_and_standardize_data(revenue_series)
    rev_per_share_normalized, _, scaler_X2 = normalize_and_standardize_data(rev_per_share_series)
    price_normalized, min_max_scaler_y, scaler_y = normalize_and_standardize_data(price_series)

    # 合并对齐后的数据作为模型输入
    X_combined = np.hstack((revenue_normalized.reshape(-1, 1), rev_per_share_normalized.reshape(-1, 1)))

    # 训练集和测试集划分
    X_train, X_test, y_train, y_test = train_test_split(X_combined, price_normalized.flatten(), test_size=0.2, random_state=42)

    # 使用 GridSearchCV 进行 alpha 参数优化
    ridge = Ridge()
    parameters = {'alpha': [0.1, 1.0, 10.0, 100.0, 200.0]}
    grid_search = GridSearchCV(ridge, parameters, scoring='neg_mean_squared_error', cv=5)
    grid_search.fit(X_train, y_train)

    # 获取最佳模型
    model = grid_search.best_estimator_

    # 预测和评估
    y_pred_final = model.predict(X_test)
    final_mse = mean_squared_error(y_test, y_pred_final)

    # 使用最新数据进行预测
    current_feature = np.array([[revenue_t3m_yoy[-1], revenue_per_share[-1]]])
    current_feature_scaled = np.hstack((
        scaler_X1.transform(current_feature[:, 0].reshape(-1, 1)),
        scaler_X2.transform(current_feature[:, 1].reshape(-1, 1))
    ))
    estimated_price_scaled = model.predict(current_feature_scaled)
    estimated_price = scaler_y.inverse_transform(estimated_price_scaled.reshape(-1, 1)).ravel()[0]

    # 计算价格差异
    price_difference = estimated_price - latest_close_price
    price_diff_percentage = price_difference / latest_close_price * 100

    if abs(price_diff_percentage) > 60:
        color = 'darkred' if latest_close_price > estimated_price else 'lightseagreen'
        action = '强力卖出' if latest_close_price > estimated_price else '强力买入'
    elif 30 <= abs(price_diff_percentage) <= 60:
        color = 'red' if latest_close_price > estimated_price else 'green'
        action = '卖出' if latest_close_price > estimated_price else '买入'
    else:
        color = 'black'
        action = ''

    result_message = (f'<span style="color: {color};">{stock_name} {stock_code} ({stock_type}) - '
                      f'实际股价: {latest_close_price:.2f}, 推算股价: {estimated_price:.2f} ({price_diff_percentage:.2f}%) {action} '
                      f'MSE: {final_mse:.2f} </span><br>')

    return result_message


In [25]:
if __name__ == "__main__":
    main()


<span style="color: black;">台泥 1101 (水泥工業) - 实际股价: 32.80, 推算股价: 27.17 (-17.16%)  MSE: 0.35 </span><br>
<span style="color: black;">亞泥 1102 (水泥工業) - 实际股价: 44.25, 推算股价: 42.70 (-3.51%)  MSE: 0.11 </span><br>
<span style="color: black;">嘉泥 1103 (水泥工業) - 实际股价: 17.95, 推算股价: 17.58 (-2.04%)  MSE: 0.20 </span><br>
<span style="color: black;">環泥 1104 (水泥工業) - 实际股价: 30.75, 推算股价: 25.38 (-17.45%)  MSE: 0.24 </span><br>
<span style="color: black;">幸福 1108 (水泥工業) - 实际股价: 16.25, 推算股价: 13.12 (-19.29%)  MSE: 0.22 </span><br>
<span style="color: black;">信大 1109 (水泥工業) - 实际股价: 18.35, 推算股价: 18.08 (-1.48%)  MSE: 0.21 </span><br>
<span style="color: black;">東泥 1110 (水泥工業) - 实际股价: 23.05, 推算股价: 18.94 (-17.85%)  MSE: 0.15 </span><br>
<span style="color: black;">味全 1201 (食品工業) - 实际股价: 18.05, 推算股价: 18.03 (-0.09%)  MSE: 0.22 </span><br>
<span style="color: red;">味王 1203 (食品工業) - 实际股价: 53.90, 推算股价: 31.63 (-41.31%) 卖出 MSE: 0.32 </span><br>
<span style="color: black;">大成 1210 (食品工業) - 实际股价: 52.10, 推算股价: 57.11 (9.61%)

KeyboardInterrupt: 